# Fire analysis of steel beam

In [1]:
import math

Check the fire capacity of an unprotected simply supported steel beam in an office building. Beam is loaded by uniformly distributed load (UDL) and it's top flange is restrained against out of plane buckling by concrete slab.

In [2]:
span = 5.9  #[m]

## Profile

In [3]:
## IPE300
h = 240 #300 #[mm]
b = 120 #150 #[mm]
tf = 15
tw = 7.1
Wply = 366600 # 628400 #[mm^4]
Iy = 38910000 # 83560000 #[mm^6]
Avz = 2568  #[mm^2]
A = 3912 #5381 #[mm^2]

## Material

In [4]:
fy = 275  #[MPa]
rho = 7850 #[kg/m^3]
gammaM0 = 1.0
E = 210000000000

## Loading

In [5]:
gammaG = 1.2
gammaQ = 1.4
psi11 = 0.5 # 0.3
# permanent loading
gk = 3.8 #[kN/m]
# variable loading
qk = 5.8 #[kN/m]
vk = gk + qk
vd = gammaG * gk + gammaQ * qk

## Check at ambient temperature

In [6]:
## Bending moment and shear force

In [7]:
MEd = vd * span **2 / 8 

In [8]:
VEd = vd * span / 2

In [9]:
# Moment capacity at ambient temperature
MplRd = Wply * fy / gammaM0

In [10]:
# Shear capacity
VplRd = 0.577 * Avz * fy / gammaM0

In [11]:
# Check deflection 
delta = 5 / 384 * vk * span**4 / E / Iy

In [12]:
if MEd <= MplRd:
    print("Moment capacity OK")
else:
    print("Moment capacity FAILs")

Moment capacity OK


In [13]:
if VEd <= VplRd:
    print("Shear capacity OK")
else:
    print("Shear capacity FAILs")

Shear capacity OK


In [14]:
if delta <= span / 250:
    print("Deflection check OK")
else:
    print("Deflection check FAILs")

Deflection check OK


## Fire check

In [15]:
# Load reduction coefficient

In [16]:
ni_fi = (gk + psi11 * qk) / (gk * gammaG + qk * gammaQ)

In [17]:
# Perimeter ratio

In [18]:
Am_by_V_b = (2 * h + b) / A * 1000

In [19]:
print ("Am_by_V_b  = " + "%2.3f" %  Am_by_V_b)

Am_by_V_b  = 153.374


In [20]:
# Shielding effect

In [21]:
Am_by_V_sh = 0.9 * Am_by_V_b

Fire resistance based on nominal fire curve (ISO 834)

Steel beam is exposed to the fire from three sides and protected from the top by concrete slab. Uneven distribution of temperature is taken in to account by coefficient

In [22]:
kappa1 = 0.7
kappa2 = 1.0
# where kappa2 expresses distribution of temperature along beam length. Cross section utilization ratio will be

In [23]:
ni0 = ni_fi * kappa1 * kappa2
ni0 = 0.202

In [24]:
print ("ni_fi      = " + "%2.3f" %  ni_fi)
print ("ni0        = " + "%2.3f" %  ni0)

ni_fi      = 0.528
ni0        = 0.202


Critical temperature (based on Cl. 4.2.4 of EN1993-1-2:2005):

In [25]:
# This is the critical temperature of the beam. We will be checking at what time is this temperature exceeded
theta_a_cr = 39.19 * math.log((1 / 0.9674 / ni0**(3.833)) - 1) + 482

In [26]:
print ("theta_a_cr = " + "%2.2f" %  theta_a_cr + " °C")

theta_a_cr = 723.48 °C


In [27]:
# init temperature
theta_at = 20
# init time
t = 0
# time increment [min]
delta_t = 0.0833

Fire resistance of unprotected steel beam can be determined as a time necessary to reach the critical temperature of a cross-section.

In [28]:
print("Time[min]  theta_g[°C]   h_net_r     h_net_c     h_net_d    ca   delta_theta_at  theta_at[°C]")
while (theta_a_cr > theta_at):
    # Specific heat of steel, ca:
    if 20 <= theta_at and theta_at < 600:
        ca = 425 + 0.773 * theta_at - 0.00169 * theta_at**2 + 0.00000222 * theta_at**3
    if 600 <= theta_at and theta_at < 735:
        ca = 666 + (13002 / (738 - theta_at))
    if 735 <= theta_at and theta_at < 900:
        ca = 545 + 17820 / (theta_at - 731)
    if 900 <= theta_at and theta_at < 1200:
        ca = 650
    
    # Heat transfer coefficient:
    alpha_c = 25 #[W/m^2/°C]
    # Heat flux (convection), gas temperature, for nominal curve (ISO834):
    
    theta_g = 20 + 345 * math.log((8 * t + 1),10)
    # Radiating temperature, can be taken equal to the gas temperature
    theta_r = theta_g
    # Surface temperature of member
    theta_m = theta_at
    # Heat flux
    h_net_c = alpha_c * (theta_g - theta_m)
    # PHI...configuration factor, usually PHI = 1.0
    PHI = 1
    # epsilon_f....emisivity of fire, usually, epsilon_f = 1.0
    epsilon_f = 1
    # epsilon_m.... emissivity of material, 0.7 for standard steel elements
    epsilon_m = 0.7
    # sigma....Stephan-Boltzman constant [W/m^2/K^4]
    sigma = 0.0000000567
    h_net_r = PHI * epsilon_m * epsilon_f * sigma * ((theta_r + 273)**4 - (theta_m + 273)**4)
    h_net_d = h_net_c + h_net_r
    # temperature increment
    delta_theta_at = 0.9 * Am_by_V_b * h_net_d * t / (ca * rho)
    theta_at = theta_at + delta_theta_at 
    print("%2.2e" %  t + "   " + "%2.3e" %  theta_g + "     " + "%2.3e" %  h_net_r + "   " + "%2.3e" %  h_net_c + "   " + "%2.2e" %  h_net_d + "   " + "%2.2e" %  ca  + "   " + "%2.2e" %  delta_theta_at + "     " + "%2.2f" %  theta_at)
    t = t + delta_t
    

    

Time[min]  theta_g[°C]   h_net_r     h_net_c     h_net_d    ca   delta_theta_at  theta_at[°C]
0.00e+00   2.000e+01     0.000e+00   0.000e+00   0.00e+00   4.40e+02   0.00e+00     20.00
8.33e-02   9.651e+01     4.474e+02   1.913e+03   2.36e+03   4.40e+02   7.86e-03     20.01
1.67e-01   1.469e+02     9.415e+02   3.173e+03   4.11e+03   4.40e+02   2.74e-02     20.04
2.50e-01   1.846e+02     1.447e+03   4.113e+03   5.56e+03   4.40e+02   5.56e-02     20.09
3.33e-01   2.146e+02     1.951e+03   4.863e+03   6.81e+03   4.40e+02   9.08e-02     20.18
4.16e-01   2.397e+02     2.448e+03   5.487e+03   7.94e+03   4.40e+02   1.32e-01     20.31
5.00e-01   2.611e+02     2.936e+03   6.020e+03   8.96e+03   4.40e+02   1.79e-01     20.49
5.83e-01   2.798e+02     3.413e+03   6.484e+03   9.90e+03   4.40e+02   2.31e-01     20.72
6.66e-01   2.965e+02     3.880e+03   6.895e+03   1.08e+04   4.40e+02   2.87e-01     21.01
7.50e-01   3.115e+02     4.336e+03   7.262e+03   1.16e+04   4.41e+02   3.47e-01     21.36
8.33e-

In [29]:
print("")
print("Time necessary to reach critical temperature in steel beam is " + str("%2.2f" % t) + " minutes." )


Time necessary to reach critical temperature in steel beam is 16.16 minutes.
